In [1]:
"""
    TODO; Test passing to labels, r_type, token strings that have multiple items separated by commas
    It should be as easy as to send to push! instead of node.labels (edge.r_type, token.token) split(node.labels, ",")
"""

include("src//graph.jl")
include("src//tokens.jl")
include("src//search.jl")

using ..HllGraph
using ..HllSets
using ..Tokens
using ..Search

using Redis
using EasyConfig
using UUIDs
using Random

conn = Redis.RedisConnection()

HllGraph.node_idx(conn, "b:nodes", "b:n")
HllGraph.edge_idx(conn,"b:edges", "b:e")

"OK"

In [2]:
# Initialize test HllSets
hll1 = HllSets.HllSet{10}()
hll2 = HllSets.HllSet{10}()
hll3 = HllSets.HllSet{10}()
hll4 = HllSets.HllSet{10}()
hll5 = HllSets.HllSet{10}()

# Generate datasets from random strings
s1 = Set(randstring(7) for _ in 1:10)
s2 = Set(randstring(7) for _ in 1:15)
s3 = Set(randstring(7) for _ in 1:100)
s4 = Set(randstring(7) for _ in 1:20)
s5 = Set(randstring(7) for _ in 1:130)

# Add datasets to HllSets
HllSets.add!(hll1, s1)
HllSets.add!(hll2, s2)
HllSets.add!(hll3, s3)
HllSets.add!(hll4, s4)
HllSets.add!(hll5, s5)

print(hll1)

HllSet{10}()


In [3]:
# Pr9nt cardinality of datasets and HllSets side by side
print(length(s1), " : ", HllSets.count(hll1), "\n")
print(length(s2), " : ", HllSets.count(hll2), "\n")
print(length(s3), " : ", HllSets.count(hll3), "\n")
print(length(s4), " : ", HllSets.count(hll4), "\n")
print(length(s5), " : ", HllSets.count(hll5), "\n\n")

# union
print(length(s1 ∪ s2 ∪ s3 ∪ s4 ∪ s5), " : ", HllSets.count(hll1 ∪ hll2 ∪ hll3 ∪ hll4 ∪ hll5), "\n")

# intersection
print(length(s1 ∩ s2 ∩ s3 ∩ s4 ∩ s5), " : ", HllSets.count(hll1 ∩ hll2 ∩ hll3 ∩ hll4 ∩ hll5), "\n")

10 : 11
15 : 14
100 : 103
20 : 22
130 : 128

275 : 277
0 : 1


In [4]:
node1 = HllGraph.Node(HllSets.id(hll1), "node1", HllSets.dump(hll1))
node2 = HllGraph.Node(HllSets.id(hll2), "node2", HllSets.dump(hll2))
node3 = HllGraph.Node(HllSets.id(hll3), "node3", HllSets.dump(hll3))
node4 = HllGraph.Node(HllSets.id(hll4), "node4", HllSets.dump(hll4))
node5 = HllGraph.Node(HllSets.id(hll5), "node5", HllSets.dump(hll5))

# println(node1.searchable)

graph = HllGraph.Graph([node1, node2, node3, node4, node5], [])

Main.HllGraph.Graph(Main.HllGraph.Node[Node(eef834d90f769df66eeefad4f800e693fe4ff413; node1; 11), Node(64d3b49ded489e3270fc584dd468be01d0f3d0b2; node2; 14), Node(14a22df49147b1d019210e05e4c5925bf22a6e62; node3; 103), Node(22c64bad7a5f32594770a6c52413dc193f5d0a62; node4; 22), Node(e9742e451a0099cee8adc5379efeedd2b646ee06; node5; 128)], Main.HllGraph.Edge[])

In [5]:
using JSON3

HllGraph.set_node(conn, node1, "b")
HllGraph.set_node(conn, node2, "b")
HllGraph.set_node(conn, node3, "b")
HllGraph.set_node(conn, node4, "b")
HllGraph.set_node(conn, node5, "b")
# println(JSON3.read(s1, Set))
Tokens.set_tokens(conn, s1, node1.sha1, "b")
Tokens.set_tokens(conn, s2, node2.sha1, "b")
Tokens.set_tokens(conn, s3, node3.sha1, "b")
Tokens.set_tokens(conn, s4, node4.sha1, "b")
Tokens.set_tokens(conn, s5, node5.sha1, "b")

In [6]:
println(HllGraph.get_node(conn, node1.sha1, "b"))
println(node1.sha1)
println(node1.labels)

nodes = HllGraph.get_nodes(conn, "b")
edges = HllGraph.get_edges(conn, "b")
println(nodes)

println(edges)

HllGraph.Graph(nodes, edges)

Node(eef834d90f769df66eeefad4f800e693fe4ff413; node1; 11)
eef834d90f769df66eeefad4f800e693fe4ff413
node1
Any[Node(eef834d90f769df66eeefad4f800e693fe4ff413; node1; 11), Node(64d3b49ded489e3270fc584dd468be01d0f3d0b2; node2; 14), Node(14a22df49147b1d019210e05e4c5925bf22a6e62; node3; 103), Node(e9742e451a0099cee8adc5379efeedd2b646ee06; node5; 128), Node(22c64bad7a5f32594770a6c52413dc193f5d0a62; node4; 22)]
Any[]


Main.HllGraph.Graph(Main.HllGraph.Node[Node(eef834d90f769df66eeefad4f800e693fe4ff413; node1; 11), Node(64d3b49ded489e3270fc584dd468be01d0f3d0b2; node2; 14), Node(14a22df49147b1d019210e05e4c5925bf22a6e62; node3; 103), Node(e9742e451a0099cee8adc5379efeedd2b646ee06; node5; 128), Node(22c64bad7a5f32594770a6c52413dc193f5d0a62; node4; 22)], Main.HllGraph.Edge[])

In [7]:
union_node = HllGraph.union_nodes(conn, node1, node2, "union", "b")

Node(ab431ec0a00c5a9227c65b00055cdd9d77ee51da; union; 24)

In [8]:
println(HllGraph.get_edges(conn, "b"))

Any[Edge(eef834d90f769df66eeefad4f800e693fe4ff413, ab431ec0a00c5a9227c65b00055cdd9d77ee51da, "union"), Edge(64d3b49ded489e3270fc584dd468be01d0f3d0b2, ab431ec0a00c5a9227c65b00055cdd9d77ee51da, "union")]


In [9]:
value ="0000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000"

println(length(value))

1024
